In [1]:
import pandas as pd
from sklearn.linear_model import Ridge
data = pd.read_csv('/Users/jsc/Downloads/player_mvp_stats.csv')

In [2]:
del data["Unnamed: 0"]
pd.isnull(data).sum()
data = data.fillna(0)
data.columns

features = [ 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS']

train = data[data["Year"] < 2021]
test = data[data["Year"] == 2021]

In [5]:
reg = Ridge(alpha=0.1)
reg.fit(train[features], train["Share"])
predictions = reg.predict(test[features])
predictions = pd.DataFrame(predictions, columns=["Predictions"], index=test.index)
predictions

,Predictions
630,0.013567
631,-0.013756
632,0.002414
633,-0.004421
634,0.010734
...,...
13897,-0.012571
13898,-0.011575
13899,0.016424
13900,-0.020434


In [22]:
combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)

combination = combination.sort_values(by="Share", ascending=False)
combination["rank"] = list(range(1, len(combination)+1))

combination = combination.sort_values(by="Predictions", ascending=False)
combination["pred_rank"] = list(range(1, len(combination)+1))
combination


,Player,Share,Predictions,rank,pred_rank
9907,Giannis Antetokounmpo,0.345,0.207436,4,1
8624,Joel Embiid,0.580,0.162713,2,2
641,Nikola Jokić,0.961,0.154307,1,3
10997,Luka Dončić,0.042,0.151430,6,4
3736,LeBron James,0.001,0.147511,15,5
...,...,...,...,...,...
9437,Patrick McCaw,0.000,-0.050344,113,536
13476,Killian Hayes,0.000,-0.051316,200,537
9915,P.J. Tucker,0.000,-0.054995,33,538
3862,Anžejs Pasečņiks,0.000,-0.062399,489,539


In [23]:
# creatign custom error metric for top 5 rankings
def avg_precision(combination):
    actual = combination.sort_values(by="Share", ascending=False).head(5)
    predicted = combination.sort_values(by="Predictions", ascending=False)
    ps = []
    found = 0
    seen = 1
    for index, row in predicted.iterrows():
        if row["Player"] in list(actual["Player"]):
            found += 1
            ps.append(found/seen)
        seen += 1
    return sum(ps)/len(ps)

In [24]:
avg_precision(combination)

0.7636363636363636